# 1000 Genomes Project TileDB-VCF Example

TileDB Cloud's Public Data Explorer provides access to a complete version of the [1000 Genomes Project](https://www.internationalgenome.org/) Phase 3 data stored as a TileDB-VCF Dataset. This tutorial provides a walkthrough of the steps we followed to create this resource using [TileDB-VCF](https://github.com/TileDB-Inc/TileDB-VCF.git)'s Python package. 

This tutorial covers:

* pre-processing combined project VCF files (pVCF) into single sample VCF files appropriate for TileDB-VCF
* creating TileDB-VCF datasets by ingesting remote VCF files stored on S3
* querying TileDB-VCF datasets via Python and accessing the results as Pandas `Dataframes`
* exporting VCF files from a TileDB-VCF dataset using the command line interface (CLI)
* scaling ingestion and query operations utilizing serverless user-defined functions (UDFs)

Please see the [official documentation](https://docs.tiledb.com/solutions/integrations/population-genomics) for more comprehensive usage details, API references, as well as instructions for installing TileDB-VCF.

## Setup and Requirements

Following along with this tutorial requires:

**System tools**
- `tiledbvcf` CLI
- `aws` command line tool
- `bcftools`
- `parallel`

**Python packages**

In [ ]:
import os
import boto3
import tiledbvcf
from envbash import load_envbash

tiledbvcf.version

The following is a custom function to simplify the process of obtaining URIs for files located on S3

In [15]:
"""
List URIs for files in an S3 bucket
:param bucket (string) Bucket name to list.
:param prefix (string) Limits the response to keys that begin with the specified prefix
:param suffix (string, optional) Limits the response to files with the specified extension
"""
def aws_s3_ls(bucket, prefix, suffix = None):
    s3 = boto3.client('s3')
    response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
    response_files = response["Contents"]
    list_incomplete = response["IsTruncated"]

    while list_incomplete:
        response = s3.list_objects_v2(
            Bucket=bucket, 
            Prefix=prefix,
            ContinuationToken = response["NextContinuationToken"]
        )
        response_files.extend(response["Contents"])
        list_incomplete = response["IsTruncated"]

    output = [f"s3://{bucket}/{file['Key']}" for file in response_files]

    if suffix is not None:
        output = [file for file in output if file.endswith(suffix)]

    return output

We'll also define define a few variables that we'll refer to throughout the tutorial. You'll need to change some of the remote locations to create your own dataset.

In [13]:
aws_bucket="genomic-datasets"
array_prefix="notebooks/1kgp3/1kgp3-array"
array_uri = f"s3://{aws_bucket}/{array_prefix}"

bcfs_prefix="1kg/1kgdbv5_supp/1kgdb_sample_bcfs"
bcf_uris = []

bedfile = "s3://genomic-datasets/notebooks/1kgp3/hg37_chr1_covidHgiGwasR4PvalC2_plog3.bed.gz"

## Preprocessing the Raw 1KG pVCF File

Unlike the more modern [high-coverage version](https://www.internationalgenome.org/announcements/3202-samples-at-high-coverage-from-NYGC/) of the 1000 Genomes (1KG) data, which provides raw single-sample gVCF files, the low-coverage Phase 3 data only provides chromosome-specific pVCF files that combine variant calls for all 2,504 samples. In order to ingest this data into a TileDB-VCF dataset we must convert the densified pVCF files into sparse single-sample VCF files.

*Note: We're only working with data from chromosome 1 for the purposes of this tutorial. However, the process of working with whole genome data is the same.*

The original VCF file was downloaded from the AWS Open Data Registry: <a href="https://registry.opendata.aws/1000-genomes" class="uri">https://registry.opendata.aws/1000-genomes</a>.

```sh
aws s3 sync \
    --exclude "*" --include "ALL.chr1.*" \
    s3://1000genomes/release/20130502/ \
    data/1000genomes/
```

First, we use `bcftools` to split the pVCF file back into single-sample VCF files

```sh
bcftools +split \
    -Ob \
    -o data/split-bcfs \
    data/1000genomes/ALL.chr1.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz
```

Next we’ll filter the split VCF files to include only records with a non-reference allele and remove `INFO` attributes that are either static (e.g., `NS`) or cohort-specific and recoverable (e.g., `AF`). We’ll save the final pre-processed files in *BCF* format, which is the binary representation of the VCF format.

```sh
rm_tags=INFO/AF,INFO/NS,INFO/EAS_AF,INFO/AMR_AF,INFO/AFR_AF,INFO/EUR_AF,INFO/SAS_AF

ls data/split-bcfs/*.bcf | parallel -j16 \
    "bcftools view --min-ac 1 -Ou -s {/.} {} | bcftools annotate -Ob --remove $rm_tags -o data/filtered-bcfs/{/}"
```

The resulting filtered BCF files are a close approximation of the raw single sample VCF files typically stored for large population genomics projects. The last step is to index the newly created BCF files.

```sh
ls data/filtered-bcfs/*.bcf | parallel -j16 "bcftools index {}"
```

For convenience, we have stored the pre-processed genome-wide single-sample VCF files on S3: 

```sh
aws s3 cp --recursive data/filtered-bcfs/ s3://{aws_bucket}/{bcfs_prefix}/
```

## Storing Data in TileDB VCF

We'll switch to Python for the rest of this tutorial and use the `tiledbvcf` package to ingest and query the pre-processed 1KG variant data

The following was run on a `m5.4xlarge` system with a 300GB EBS volume to handle the large number of VCF files. Note that TileDB is *highly-tunable* and while the defaults were chosen to provide a good balance between ingestion speed, read performance, and dataset size, they can be be tweaked to better suit a specific use case.

### Create the dataset

You can create a TileDB VCF dataset anywhere that TileDB supports, this could be a local filesystem, S3, Azure, Google Cloud Storage, HDFS, and more. For this example we’ll create it on S3, the most common use case.

We're opening the dataset in *write* mode, with `verbose = True` to receive information about the progress of the ingestion, and `stats = True` to get some insight into TileDB's performance after ingestion completes.

In [ ]:
ds = tiledbvcf.Dataset(array_uri, stats = True, verbose = True, mode = "w")
ds.create_dataset(extra_attrs = ["fmt_GT"])

#### Storing the data

Storing VCF data in TileDB-VCF simply requires a list of the VCF/BCF file locations. In this case, we'll provide a list of S3 URIS pointing to the BCF files, which will allow us to ingest them directly from their remote location.

In [50]:
bcf_uris = aws_s3_ls(aws_bucket, bcfs_prefix, suffix = "bcf")

Finally, we'll run the following command to ingest the pre-processed BCF files into a new TileDB-VCF dataset.

In [ ]:
ds.ingest_samples(
    sample_uris = bcf_uris,
    threads = 14,
    memory_budget_mb = 2048 * 2,
    sample_batch_size = 20,
    scratch_space_path = "/mnt/data/tmp",
    scratch_space_size = 4000
)

We can see from the verbose output provided the following summary printed at the end:

```
All finalize tasks successfully completed. Waited for 5.87609 sec.
Done. Ingested 10,771,609,147 records (+ 38,528,651 anchors) from 2,504 samples in 24,546.9 seconds.
```

This indicates we’ve ingested over 10 billion records into the TileDB-VCF dataset in just under 7 hours. With an `m5.4xlarge` instance costing \$0.768 an hour, the cost of ingestion was just over \$5.00 USD.

The final array is 70Gb, or just under half the size of the individual compressed BCF files.

In [9]:
print(ds.tiledb_stats())


==== WRITE ====

- Number of write queries: 70434

- Number of attributes written: 633906
  * Number of fixed-sized attributes written: 211302
  * Number of var-sized attributes written: 422604
- Number of dimensions written: 211302
  * Number of fixed-sized dimensions written: 70434
  * Number of var-sized dimensions written: 140868

- Number of bytes written: 75362471867 bytes (70.1868 GB) 
- Number of write operations: 21863256
- Number of bytes filtered: 2278398374656 bytes (2121.92 GB) 
- Filtering deflation factor: 30.2325x

- Total metadata written: 89454680 bytes (0.0833112 GB) 
  * Array schema: 813 bytes (7.57165e-07 GB) 
  * Fragment metadata footer: 1605894 bytes (0.00149561 GB) 
  * R-tree: 7523989 bytes (0.00700726 GB) 
  * Fixed-sized tile offsets: 38844555 bytes (0.0361768 GB) 
  * Var-sized tile offsets: 28188665 bytes (0.0262527 GB) 
  * Var-sized tile sizes: 13290764 bytes (0.012378 GB) 

- Time to write array metadata: 0.156775 secs
  * Array metadata size: 110 byt

Following ingestion, it may help performance to consolidate the metadata fragments, which is currently only possible using the CLI.

```
```

In [ ]:
!tiledbvcf utils consolidate fragment_meta -u {array_uri}

## Reading, Analysis, and Exporting

In this section we will walk through accessing the TileDB-VCF dataset and also exporting back to VCF and TSV. First, we must reopen the dataset in *read* mode.


In [5]:
ds = tiledbvcf.Dataset(array_uri, stats = True, verbose = True, mode = "r")

#### Reading into Pandas Dataframe

Pandas is one of the most popular data science tools in python. TileDB VCF’s python API produces results directly as Pandas `DataFrame`s, making it easy to analyze the data and leverage any of Pandas’ builtin algorithms.

Let’s run a typical query on the 1kg TileDB-VCF dataset we created above. We’ll retrieve all variants that overlap the gene *MTOR* on chr 1 for sample HG00096, along with a few attributes.

In [16]:
cfg = tiledbvcf.ReadConfig(memory_budget_mb=8192)
ds = tiledbvcf.Dataset(array_uri, stats = True, cfg = cfg)

ds.read(
    attrs = ["sample_name", "contig", "pos_start", "pos_end", "alleles", "fmt_GT"], 
    regions = ["1:43337848-43352772"],
    samples = ["HG00096"]
)

,sample_name,contig,pos_start,pos_end,alleles,fmt_GT
0,HG00096,1,43337897,43337897,"[A, G]","[1, 0]"
1,HG00096,1,43339092,43339092,"[C, T]","[1, 1]"
2,HG00096,1,43339203,43339203,"[G, A]","[1, 1]"
3,HG00096,1,43340776,43340776,"[T, C]","[1, 1]"
4,HG00096,1,43340779,43340779,"[A, G]","[1, 1]"
5,HG00096,1,43341662,43341662,"[A, G]","[1, 1]"
6,HG00096,1,43342021,43342021,"[G, A]","[1, 0]"
7,HG00096,1,43343390,43343390,"[A, G]","[1, 1]"
8,HG00096,1,43344193,43344193,"[G, A]","[1, 1]"
9,HG00096,1,43344632,43344632,"[T, A]","[1, 1]"


If the `Dataset` object was created with `stats = True` you can print out a variety of useful information about the query:

In [7]:
print(ds.tiledb_stats())

==== READ ====

- Number of read queries: 5
- Number of attempts until results are found: 5

- Number of attributes read: 6
  * Number of fixed-sized attributes read: 2
  * Number of var-sized attributes read: 4
- Number of dimensions read: 7
  * Number of fixed-sized dimensions read: 1
  * Number of var-sized dimensions read: 6

- Number of logical tiles overlapping the query: 255
- Number of physical tiles read: 5865
  * Number of physical fixed-sized tiles read: 765
  * Number of physical var-sized tiles read: 5100
- Number of cells read: 15048
- Number of result cells: 5029
- Percentage of useful cells read: 33.4197%

- Number of bytes read: 1640652 bytes (0.00152798 GB) 
- Number of read operations: 4389
- Number of bytes unfiltered: 5584858 bytes (0.0052013 GB) 
- Unfiltering inflation factor: 3.40405x

- Time to compute estimated result size: 0.984495 secs
  * Time to compute tile overlap: 1.0006 secs
    > Time to compute relevant fragments: 0.00160453 secs
    > Time to load r

For production-sized queries that encompass large portions of the genome it's more convenient to provide bed files with the query regions. Here, we'll use a bed file on S3 that contains 1,040 regions on chr1 that show at least a moderate association with with SARS-CoV-2 infection susceptibility (data obtained from the [COVID-19 Host Genetics Initiative](https://www.covid19hg.org/)).

In [17]:
ds = tiledbvcf.Dataset(array_uri, stats = True, cfg = cfg)

df = ds.read(
    attrs = ["sample_name", "contig", "pos_start", "pos_end", "alleles", "info_DP", "fmt_GT"], 
    samples = ds.samples()[:10],
    bed_file = bedfile
)

df

,sample_name,contig,pos_start,pos_end,alleles,info_DP,fmt_GT
0,HG00097,1,2265099,2265099,"[C, T]",[16479],"[0, 1]"
1,HG00099,1,2265099,2265099,"[C, T]",[16479],"[0, 1]"
2,HG00107,1,2265099,2265099,"[C, T]",[16479],"[1, 0]"
3,HG00096,1,2337537,2337537,"[C, G]",[11721],"[1, 1]"
4,HG00097,1,2337537,2337537,"[C, G]",[11721],"[1, 1]"
...,...,...,...,...,...,...,...
3470,HG00102,1,247999680,247999680,"[G, A]",[19889],"[1, 1]"
3471,HG00103,1,247999680,247999680,"[G, A]",[19889],"[1, 1]"
3472,HG00105,1,247999680,247999680,"[G, A]",[19889],"[1, 1]"
3473,HG00106,1,247999680,247999680,"[G, A]",[19889],"[1, 1]"


This query completed in 1.9 secs for 10 samples.

#### Filter Example

Using the pandas dataframe returned from TileDB-VCF we can apply additional filters. For instance if we wanted to filter the above result on read depth (`info_DP`):

In [ ]:
df[df.info_DP.apply(lambda x: x[0] > 5000)]

The python API supports a variety of advanced uses: batching, partitioning, Dask, and more. We are happy to follow-up with additional details beyond these initial examples.

### CLI Exporting

In addition to the python API it is also possible to export the dataset back into VCF format. This can be helpful in interoperating with legacy tools.

#### Exporting to VCF

When exporting to VCF you can specify any number of samples, and each will be exported to its own file in vcf, compressed vcf or bcf format depdning on what you set for `--output-format`.

For example to export the entire sample for `HG00096`, `HG00097`, and `HG00099` you can run the following:

In [ ]:
!tiledbvcf export --uri {array_uri} \
    --output-format v \
    --sample-names HG00096,HG00097,HG00099 \
    --verbose

This produces 3 files: `HG00096.vcf`, `HG00097.vcf`, and `HG00099.vcf`.

##### Filtering Exports

You can also combine the use of regions (bed file or list of regions passed to cli) to filter the export.

In [ ]:
!tiledbvcf export --uri {array_uri} \
    --output-format v \
    --sample-names HG00096,HG00097,HG00099 \
    --regions-file {bedfile} \
    --verbose

This will also produce the 3 VCF files, like the previous export. However, these files are filtered for the same SARS-CoV-2 associated genomic regions specified in the bed file.


#### Exporting to TSV

For even more generic usecases you can export data to tab seperate files with the `--output-format t` option.

In [ ]:
!tiledbvcf export --uri {array_uri} \
    --output-format t \
    --tsv-fields CHR,POS,I:END,REF,ALT,S:GT,Q:POS,Q:END \
    --sample-names HG00096,HG00097,HG00099 \
    --regions-file {bedfile} \
    --verbose --output-path sars-cov-2-associated-regions.tsv

## Scaling with Severless Computer

### UDF-based Ingestion

Now that we've seen the basic mechanics of how to create and query a TileDB-VCF dataset, we'll look at some options for solving the most common pain point in modern genomics analyses: scaling operations to process the massive cohorts produced by modern sequencing projects.

Out of the box, TileDB supports fully parallel reads and writes so queries can be partitioned based on genomic regions, samples, or both, and distributed across multiple cores, with each core processing a different chunk of the data. TileDB-VCF provides a number of options to leverage these scaling features, including integrations with frameworks like Dask for parallel computing in Python, as well as Apache Spark.

Here, we'll utilize TileDB Cloud's serverless compute service to easily convert our previous single-node ingestion into one that's automatically processed by as many nodes as we have sample batches. First, we'll need to import `tiledb.cloud`, which provides the `Delayed` module we'll use to convert our query function into a user-defined function (UDF) that can be serialized and shipped to serverless node for processing.


In [ ]:
from tiledb.cloud.compute import Delayed

Then we'll need to create a callable function that performs our ingestion for a given batch of samples.

In [79]:
def ingest_bcf_files(array_uri, bcf_uris, cfg):
    print(f"Ingesting {len(bcf_uris)} starting with {bcf_uris[0]}")
    ds = tiledbvcf.Dataset(array_uri, mode = "w", cfg = cfg)
    ds.ingest_samples(sample_uris = bcf_uris, threads = 2, memory_budget_mb = 512)
    return bcf_uris

This function requires 3 inputs: a URI for the array, a list of URIs for the BCF files, and a dictionary containing the necessary AWS parameters. It returns the same list of BCF URI's simply to pass them to a second function that collects the same output from all of the parallel UDFs to serve as a root dependency task.

In [83]:
def return_samples(file_list):
    out = []
    [out.extend(i) for i in file_list]
    return out

Next, we'll split our list of BCF URIs into groups, each of which represents a single UDF task. It is recommended that the group sizes are equal to the ingestion `sample_batch_size` paramter to avoid creating more fragments than necessary. 

In [ ]:
n = 20
batched_uris = [bcf_uris[i * n:(i + 1) * n] for i in range((len(bcf_uris) + n - 1) // n )]

Just as we did above, we're going to create a new dataset to ingest our BCF files into, the only difference being we need to pass along our AWS credenetials to grant the nodes access. 

In [ ]:
array_uri2 = f"s3://{aws_bucket}/{array_prefix}_udf"

cfg = tiledbvcf.ReadConfig(tiledb_config = {
    "vfs.s3.aws_access_key_id": os.getenv("TILEDB_VFS_S3_AWS_ACCESS_KEY_ID"),
    "vfs.s3.aws_secret_access_key": os.getenv("TILEDB_VFS_S3_SECRET_ACCESS_KEY")
})

ds = tiledbvcf.Dataset(uri = array_uri2, mode = "w", stats = True, verbose = True, cfg = cfg)
ds.create_dataset(extra_attrs = ["info_GT"])

Finally, we'll create delayed versions of this functions populated with the necessary argument values and then execute the ingestion by calling `compute()`.

In [ ]:
delayed_writes = [Delayed(ingest_bcf_files)(array_uri, b, cfg) for b in batched_uris]
ingested_samples = Delayed(return_samples, name = "Combine")(delayed_writes)

ingested_samples.compute()

### UDF-based Queries

We can follow a very similar process to parallelize queries. First, we'll need to create a callable function that performs our query for a specific sample:

In [ ]:
def query_vcf_dataset(sample):
    ds = tiledbvcf.Dataset(array_uri, mode = "r")
    return ds.read(attrs, samples = [sample], bed_file = bed_file)

We then need to create a *delayed* instance of this `query_vcf_dataset()` function for each sample in our dataset.

In [ ]:
sample_dfs = [Delayed(query_vcf_dataset)(sample = s) for s in ds.samples()[0:99]]

After execution, sample_dfs will comprise a list of dataframes, each containing the sample-specific results. Let's add one more UDF that will combine the individual dataframes into a single result.

In [ ]:
def combine_results(df_list):
    return pd.concat(df_list)

combined_df = Delayed(combine_results)(sample_dfs)


At this point, we have constructed a UDF task graph containing 10 query tasks that will be performed in parallel and a merge step to combine the results. Let's visualize the task graph to verify the number of tasks and inter-dependencies look correct:

In [ ]:
combined_df.visualize(force_plotly = True)

Finally, let's execute our serverless distributed queries. If you're running this tutorial interactively, keep an eye on the diagram above. The nodes are color coded to indicate the current status of each task and update in real time.

In [ ]:
combined_df.compute()